In [1]:
import pandas as pd
import requests

from fcd_torch import FCD
import torch

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

import numpy as np
import time
import GPUtil

In [13]:
filename = '/mnt/usb/cmdunham/BKG_SIM_ims_acbc_train_v1.1.09_meta.feather'
metadata = pd.read_feather(filename)

filename = '/mnt/usb/cmdunham/BKG_SIM_ims_acbc_train_v1.1.09_spectra.feather'
spectra = pd.read_feather(filename)

# Getting Simulant Names/SMILES:
---

In [3]:
labels = list(metadata['Label'].unique())
labels

['BKG', 'DEM', 'DEB', 'MES', 'DMMP', 'DPM', 'TEPO', 'JP8', 'DtBP']

In [ ]:
simulant_name_mapping = {'BKG':{'Name':'Background', 'SMILES':'NA'}, 'DEM':{'Name': 'Diethyl Malonate'}, 'DEB':{'Name': '1,2,3,4-Diepoxybutane' }, 
                         'MES':{'Name': '2-(N-morpholino)ethanesulfonic acid'}, 'DMMP':{'Name': 'Dimethyl methylphosphonate'}, 'DPM':{'Name': 'Oxybispropanol'}, 
                         'JP8':{'Name': 'JP8'}, 'TEPO':{'Name': 'Triethyl phosphate', 'SMILES': 'CCOP(=O)(OCC)OCC'},'DtBP':{'Name': 'Di-tert-butyl peroxide', 'SMILES': 'O(OC(C)(C)C)C(C)(C)C'}}
names = []
for key in simulant_name_mapping:
    names.append(simulant_name_mapping[key]['Name'])


In [3]:
for abbreviation in simulant_name_mapping:
    name = simulant_name_mapping[abbreviation]['Name']
    try:
        # send simulant name to PubChem's API and get SMILES out of response
        url = (f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{name}/property/CanonicalSMILES/JSON')
        r = requests.get(url).json()
        smiles = r['PropertyTable']['Properties'][0]['CanonicalSMILES']
        simulant_name_mapping[abbreviation]['SMILES'] = smiles

    except KeyError as e:
        try:
            # send simulant abbreviation to PubChem's API and get SMILES out of response
            url = (f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{abbreviation}/property/CanonicalSMILES/JSON')
            r = requests.get(url).json()
            smiles = r['PropertyTable']['Properties'][0]['CanonicalSMILES']
            simulant_name_mapping[abbreviation]['SMILES'] = smiles
        except KeyError as e:
            if e == 'PropertyTable':
                simulant_name_mapping[abbreviation]['SMILES'] = 'unknown'

simulant_name_mapping

{'BKG': {'Name': 'Background', 'SMILES': 'NA'},
 'DEM': {'Name': 'Diethyl Malonate', 'SMILES': 'CCOC(=O)CC(=O)OCC'},
 'DEB': {'Name': '1,2,3,4-Diepoxybutane', 'SMILES': 'C1C(O1)C2CO2'},
 'MES': {'Name': '2-(N-morpholino)ethanesulfonic acid',
  'SMILES': 'C1COCCN1CCS(=O)(=O)O'},
 'DMMP': {'Name': 'Dimethyl methylphosphonate', 'SMILES': 'COP(=O)(C)OC'},
 'DPM': {'Name': 'Oxybispropanol', 'SMILES': 'CC(CO)OC(C)CO.CC(COCC(C)O)O'},
 'JP8': {'Name': 'JP8', 'SMILES': 'C1CC(NC1)COC(=O)NC2CCC(=O)NC2=O'},
 'TEPO': {'Name': 'Triethyl phosphate', 'SMILES': 'CCOP(=O)(OCC)OCC'},
 'DtBP': {'Name': 'Di-tert-butyl peroxide', 'SMILES': 'CC(C)(C)OOC(C)(C)C'}}

# Getting Simulant ChemNet Embeddings:
---

In [ ]:
# Get ChemNet embeddings from simulant SMILES
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
fcd = FCD(device, n_jobs=1)
for abbreviation in simulant_name_mapping:
    if abbreviation != 'BKG':
        smiles = simulant_name_mapping[abbreviation]['SMILES']
        simulant_name_mapping[abbreviation]['embedding'] = list(fcd.get_predictions([smiles])[0])

# Saving Background, Spectra and Embedding Files:
---

## Background:
---

In [7]:
# Remove all simulant spectra, leave only background
bkg = spectra[spectra['Label'] == 'BKG']
bkg.head()

,index,p_184,p_185,p_186,p_187,p_188,p_189,p_190,p_191,p_192,...,n_1013,n_1014,n_1015,n_1016,n_1017,n_1018,n_1019,n_1020,n_1021,Label
0,1427,92.0,94.0,98.0,101.0,106.0,110.0,117.0,121.0,127.0,...,-11.0,-11.0,-11.0,-8.0,-8.0,-7.0,-7.0,-7.0,-7.0,BKG
1,1452,85.0,87.0,92.0,98.0,103.0,110.0,116.0,123.0,127.0,...,-13.0,-14.0,-17.0,-18.0,-19.0,-20.0,-17.0,-17.0,-17.0,BKG
2,1467,86.0,91.0,97.0,103.0,109.0,115.0,120.0,125.0,130.0,...,-18.0,-18.0,-22.0,-22.0,-24.0,-23.0,-23.0,-23.0,-21.0,BKG
3,1486,97.0,98.0,102.0,104.0,107.0,110.0,114.0,117.0,121.0,...,-27.0,-26.0,-26.0,-22.0,-21.0,-22.0,-23.0,-23.0,-27.0,BKG
4,1564,91.0,96.0,101.0,105.0,112.0,117.0,121.0,125.0,128.0,...,-14.0,-17.0,-18.0,-21.0,-21.0,-19.0,-18.0,-17.0,-16.0,BKG


In [8]:
# file_path = '/mnt/usb/cmdunham/preprocessed_ims_data/bkg.csv'
# bkg.to_csv(file_path, index=False)

## Spectra:
---

In [14]:
# Remove all background spectra, leave only simulants
simulants = spectra[spectra['Label'] != 'BKG']

# Create OneHot encodings of chem abbreviations
enc = OneHotEncoder(sparse_output=False)
encodings = enc.fit_transform(simulants[['Label']])

# get chemical abbreviations from enc.categories
columns = [abbr for abbr in enc.categories_[0]]
# create encoding df 
encoding_df = pd.DataFrame(encodings, columns=columns)

In [15]:
# dropping indices so concatenation works correctly  
simulants.reset_index(drop=True, inplace=True)
encoding_df.reset_index(drop=True, inplace=True)

# concatenate simulants and encoding df
spectra_and_encodings = pd.concat([simulants, encoding_df], axis=1)
spectra_and_encodings.head()

,index,p_184,p_185,p_186,p_187,p_188,p_189,p_190,p_191,p_192,...,n_1021,Label,DEB,DEM,DMMP,DPM,DtBP,JP8,MES,TEPO
0,316161,99.0,103.0,106.0,111.0,116.0,121.0,125.0,130.0,136.0,...,-26.0,MES,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,316163,83.0,87.0,92.0,95.0,100.0,104.0,109.0,114.0,120.0,...,8.0,MES,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,316164,91.0,95.0,97.0,101.0,107.0,113.0,120.0,128.0,134.0,...,7.0,MES,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,316165,87.0,91.0,95.0,99.0,103.0,107.0,111.0,119.0,126.0,...,-13.0,MES,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,316166,75.0,79.0,82.0,87.0,94.0,100.0,107.0,112.0,118.0,...,-2.0,MES,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [19]:
# divide into train and test data
train_val, test = train_test_split(spectra_and_encodings, test_size=.2, random_state = 22)
# divide train into train and validation data
train, val = train_test_split(train_val, test_size=.25, random_state=22)

print(train.shape, val.shape, test.shape)

(222519, 1686) (74173, 1686) (74173, 1686)


In [17]:
file_path = '/mnt/usb/cmdunham/preprocessed_ims_data/train_data.csv'
train.to_csv(file_path)

file_path = '/mnt/usb/cmdunham/preprocessed_ims_data/val_data.csv'
val.to_csv(file_path)

file_path = '/mnt/usb/cmdunham/preprocessed_ims_data/test_data.csv'
test.to_csv(file_path)

## Embeddings:
---

In [12]:
name_smiles_embedding_df = pd.DataFrame(simulant_name_mapping).T
name_smiles_embedding_df.head()

,Name,SMILES,embedding
BKG,background,NA,NaN
DEM,Diethyl Malonate,CCOC(=O)CC(=O)OCC,"[0.3809721, 0.0005454041, 0.25539744, -0.24272..."
DEB,"1,2,3,4-Diepoxybutane",C1C(O1)C2CO2,"[0.06318794, 0.009022224, 0.42160064, 0.195722..."
MES,2-(N-morpholino)ethanesulfonic acid,C1COCCN1CCS(=O)(=O)O,"[-0.32520828, 0.009838344, -0.15108332, 0.2845..."
DMMP,Dimethyl methylphosphonate,COP(=O)(C)OC,"[0.12106811, 0.00294244, -0.14450458, 0.072665..."


In [ ]:
# # save df with chem names, smiles and ChemNet embeddings
# file_path = '/mnt/usb/cmdunham/preprocessed_ims_data/name_smiles_embedding_file.csv'
# name_smiles_embedding_df.to_csv(file_path)

# Generating Carls:
---

In [17]:
if torch.cuda.is_available():
    # Get the list of GPUs
    gpus = GPUtil.getGPUs()

    # Find the GPU with the most free memory
    best_gpu = max(gpus, key=lambda gpu: gpu.memoryFree)

    # Print details about the selected GPU
    print(f"Selected GPU ID: {best_gpu.id}")
    print(f"  Name: {best_gpu.name}")
    print(f"  Memory Free: {best_gpu.memoryFree} MB")
    print(f"  Memory Used: {best_gpu.memoryUsed} MB")
    print(f"  GPU Load: {best_gpu.load * 100:.2f}%")

    # Set the device for later use
    device = torch.device(f'cuda:{best_gpu.id}')
    print('Current device ID: ', device)

    # Set the current device in PyTorch
    torch.cuda.set_device(best_gpu.id)
else:
    device = torch.device('cpu')
    print('Using CPU')

# Confirm the currently selected device in PyTorch
print("PyTorch current device ID:", torch.cuda.current_device())
print("PyTorch current device name:", torch.cuda.get_device_name(torch.cuda.current_device()))

Selected GPU ID: 0
  Name: NVIDIA GeForce RTX 4090
  Memory Free: 15626.0 MB
  Memory Used: 8590.0 MB
  GPU Load: 67.00%
Current device ID:  cuda:0
PyTorch current device ID: 0
PyTorch current device name: NVIDIA GeForce RTX 4090


In [18]:
file_path = '/mnt/usb/cmdunham/preprocessed_ims_data/bkg.csv'
background = pd.read_csv(file_path)

file_path = '/mnt/usb/cmdunham/preprocessed_ims_data/train_data.csv'
train = pd.read_csv(file_path)

file_path = '/mnt/usb/cmdunham/preprocessed_ims_data/val_data.csv'
val = pd.read_csv(file_path)

file_path = '/mnt/usb/cmdunham/preprocessed_ims_data/test_data.csv'
test = pd.read_csv(file_path)

In [19]:
n=2
background_sample = background.sample(n=n, random_state=42).iloc[:,1:-1]
background_sample.reset_index(inplace=True)
background_sample.drop(columns=['index'], inplace=True)

In [20]:
def generate_differences_df(spectra_df, background_sample):
    # Convert DataFrames to PyTorch tensors
    spectra = spectra_df.iloc[:,2:-9]
    tensor1 = torch.tensor(spectra.values, dtype=torch.float32)
    tensor2 = torch.tensor(background_sample.values, dtype=torch.float32)

    # Move tensors to GPU if available
    if torch.cuda.is_available():
        tensor1 = tensor1.cuda()
        tensor2 = tensor2.cuda()

    differences = tensor1.unsqueeze(1) - tensor2.unsqueeze(0)

    differences_np = differences.cpu().numpy()  # move differences to cpu

    # Reshape the differences to 2D for DataFrame creation
    m, p, n = differences_np.shape
    differences_reshaped = differences_np.reshape(m * p, n)

    # Create a DataFrame from the reshaped array
    differences_df = pd.DataFrame(differences_reshaped)

    index_col = np.repeat(spectra_df['index'].values, p)
    differences_df.insert(0, 'index', index_col)
    label_info = pd.DataFrame(np.repeat(spectra_df.iloc[:,-9:].values, 2, axis=0))
    differences_df = pd.concat([differences_df, label_info], axis=1)
    differences_df.columns = spectra_df.columns[1:]

    return differences_df

In [21]:
file_path='/mnt/usb/cmdunham/preprocessed_ims_data/train_carls.csv'
differences_df = generate_differences_df(train, background_sample)
differences_df.to_csv(file_path, index=False, header=True)

file_path='/mnt/usb/cmdunham/preprocessed_ims_data/val_carls.csv'
differences_df = generate_differences_df(val, background_sample)
differences_df.to_csv(file_path, index=False, header=True)

file_path='/mnt/usb/cmdunham/preprocessed_ims_data/test_carls.csv'
differences_df = generate_differences_df(test, background_sample)
differences_df.to_csv(file_path, index=False, header=True)

KeyboardInterrupt: 

# Unused Code:
---

In [ ]:
# Convert DataFrames to PyTorch tensors
# train_sample = train.sample(n=100000, random_state=42).iloc[:,2:-9]
train_spectra = train.iloc[:,2:-9]
tensor1 = torch.tensor(train_spectra.values, dtype=torch.float32)
tensor2 = torch.tensor(background_sample.values, dtype=torch.float32)

# Move tensors to GPU if available
if torch.cuda.is_available():
    tensor1 = tensor1.cuda()
    tensor2 = tensor2.cuda()

differences = tensor1.unsqueeze(1) - tensor2.unsqueeze(0)
differences.shape

torch.Size([267022, 2, 1676])

In [ ]:
differences_np = differences.cpu().numpy()  # move differences to cpu

# Reshape the differences to 2D for DataFrame creation
m, p, n = differences_np.shape
differences_reshaped = differences_np.reshape(m * p, n)

# Create a DataFrame from the reshaped array
differences_df = pd.DataFrame(differences_reshaped)

In [ ]:
differences_df.columns = train.columns[2:-9]
index_col = np.repeat(train['index'].values, p)
differences_df.insert(0, 'index', index_col)
differences_df = pd.concat([differences_df, train.iloc[:,-9:]], axis=1)
differences_df.head()

,index,p_184,p_185,p_186,p_187,p_188,p_189,p_190,p_191,p_192,...,n_1021,Label,DEB,DEM,DMMP,DPM,DtBP,JP8,MES,TEPO
0,1297539,-12.0,-14.0,-16.0,-20.0,-21.0,-22.0,-22.0,-24.0,-25.0,...,-8.0,DtBP,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1297539,-12.0,-10.0,-7.0,-4.0,1.0,3.0,7.0,6.0,8.0,...,15.0,DMMP,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1344903,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,-2.0,...,-7.0,TEPO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1344903,2.0,6.0,11.0,17.0,24.0,27.0,30.0,31.0,31.0,...,16.0,DtBP,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1221071,10.0,12.0,15.0,14.0,17.0,19.0,22.0,23.0,26.0,...,0.0,TEPO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
# def make_carl_file(data, background_spectra, file_path, n):
#     columns = list(data.columns[1:])
#     with open(file_path, mode='w', newline='') as file:
#         header_written = False
#         for idx, row in data.iterrows():
#             if idx < 3:
#                 indices = []
#                 diffs = []
#                 labels_encodings = []
#                 simulant_array = row[2:-9].values
#                 for i in range(n):
#                     background_array = background_spectra.iloc[i].values
#                     diff = simulant_array - background_array
#                     # add index, label and label encodings back in
#                     # diff = np.concatenate([[row['index']], diff, row[-9:].values])
#                     indices.append(row['index'])
#                     diffs.append(diff)
#                     labels_encodings.append(row[-9:].values)
#                 # if idx % 10 == 0:
#                 # Create a DataFrame for the current row
#                 diffs_df = pd.DataFrame(diffs)
                
#                 carl_df_chunk = pd.DataFrame(diffs_df, columns=columns)

#                 # Write chunk to CSV
#                 if not header_written:
#                     carl_df_chunk.to_csv(file, index=False, header=True)
#                     header_written = True
#                 else:
#                     carl_df_chunk.to_csv(file, index=False, header=False)

In [ ]:
def make_carl_file(data, background_spectra, file_path, n):
    columns = list(data.columns[1:])
    with open(file_path, mode='w', newline='') as file:
        header_written = False
        for idx, row in data.iterrows():
            if idx < 3:
                diffs = []
                simulant_array = row[2:-9].values
                for i in range(n):
                    background_array = background_spectra.iloc[i].values
                    diff = simulant_array - background_array
                    # add index, label and label encodings back in
                    diff = np.concatenate([[row['index']], diff, row[-9:].values])
                    diffs.append(diff)
                # Create a DataFrame for the current row                
                carl_df_chunk = pd.DataFrame(diffs, columns=columns)

                # Write chunk to CSV
                if not header_written:
                    carl_df_chunk.to_csv(file, index=False, header=True)
                    header_written = True
                else:
                    carl_df_chunk.to_csv(file, index=False, header=False)

In [59]:
# def make_carl_file(data, background_spectra, file_path, n):
#     diffs = []
#     for idx, row in data.iterrows():
#         if idx < 10:
#             simulant_array = row[2:-9].values
#             for i in range(n):
#                 background_array = background_spectra.iloc[i].values
#                 diff = simulant_array - background_array
#                 # add index, label and label encodings back in
#                 diff = np.concatenate([[row['index']], diff, row[-9:].values])
#                 diffs.append(diff)

#     columns = list(data.columns[1:])
#     carl_df = pd.DataFrame(diffs, columns=columns)
#     carl_df.to_csv(file_path, index=False)

In [102]:
# start_time = time.time()    
# n = 2
# make_carl_file(train, background_sample, file_path='/mnt/usb/cmdunham/preprocessed_ims_data/train_carls.csv', n=n)
# # make_carl_file(val, background_sample, file_path='/mnt/usb/cmdunham/preprocessed_ims_data/val_carls.csv', n=n)
# # make_carl_file(test, background_sample, file_path='/mnt/usb/cmdunham/preprocessed_ims_data/test_carls.csv', n=n)

# end_time = time.time()
# print('Total run time: ', end_time - start_time)

In [103]:
file_path = '/mnt/usb/cmdunham/preprocessed_ims_data/val_carls.csv'
carls = pd.read_csv(file_path)
carls.head()

,index,p_184,p_185,p_186,p_187,p_188,p_189,p_190,p_191,p_192,...,n_1021,Label,DEB,DEM,DMMP,DPM,DtBP,JP8,MES,TEPO
0,655550,-9.0,-11.0,-15.0,-22.0,-26.0,-29.0,-32.0,-35.0,-39.0,...,-7.0,DEB,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,384167,-9.0,-10.0,-9.0,-12.0,-13.0,-14.0,-17.0,-21.0,-25.0,...,-9.0,DEM,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1141374,-19.0,-19.0,-20.0,-23.0,-25.0,-28.0,-32.0,-37.0,-42.0,...,-12.0,DMMP,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1317186,-1.0,0.0,2.0,2.0,3.0,6.0,6.0,7.0,5.0,...,-5.0,TEPO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1110674,-26.0,-27.0,-28.0,-28.0,-30.0,-31.0,-34.0,-39.0,-44.0,...,-1.0,TEPO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
